# ETL – Camada Raw → Silver (Limpeza e Padronização)

## Visão Geral

Este processo tem como objetivo preparar os dados brutos do **Airbnb Austin** para análise, por meio de limpeza, padronização e correção de tipos de dados.

O resultado é a camada **Silver**, contendo dados consistentes, confiáveis e prontos para uso analítico.



In [ ]:
import re
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Caminhos relativos (Transformer/ ../raw/ e ../silver/)
RAW_DIR = Path('..') / 'raw'
SILVER_DIR = Path('..') / 'silver'
SILVER_DIR.mkdir(exist_ok=True)

print('=' * 70)
print('ETL: RAW → SILVER - LIMPEZA E PADRONIZAÇÃO DE DADOS')
print('=' * 70)
print(f'Data de execução: {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}')
print(f'RAW_DIR = {RAW_DIR.resolve()}')
print(f'SILVER_DIR = {SILVER_DIR.resolve()}')
print('=' * 70)

ETL: RAW → SILVER - LIMPEZA E PADRONIZAÇÃO DE DADOS
Data de execução: 11/01/2026 20:25:46
RAW_DIR = /home/mateus/Área de trabalho/airbnb/SBD2-Austin-Airbnb/Data Layer/raw
SILVER_DIR = /home/mateus/Área de trabalho/airbnb/SBD2-Austin-Airbnb/Data Layer/silver


In [ ]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS DE LIMPEZA
# =============================================================================

def converter_para_snake_case(nome_coluna):
    """
    Converte nomes de colunas para snake_case.
    Exemplo: 'Host Name' → 'host_name'
    """
    # Remove espaços extras
    nome = nome_coluna.strip()
    # Substitui espaços e hífens por underscore
    nome = re.sub(r'[\s\-]+', '_', nome)
    # Remove caracteres especiais
    nome = re.sub(r'[^a-z0-9_]', '', nome.lower())
    # Remove underscores múltiplos
    nome = re.sub(r'_+', '_', nome)
    return nome.strip('_')

def limpar_texto(texto):
    """
    Remove espaços extras, quebras de linha e caracteres de controle.
    """
    if pd.isna(texto):
        return texto
    texto = str(texto).strip()
    # Remove tabs, quebras de linha
    texto = re.sub(r'[\r\n\t]+', ' ', texto)
    # Remove espaços duplos
    texto = re.sub(r'\s+', ' ', texto)
    return texto

def converter_preco(valor):
    """
    Converte string de preço para float.
    Exemplos: '$1,234.56' → 1234.56, '' → NaN
    """
    if pd.isna(valor):
        return np.nan
    valor_str = str(valor).strip()
    if valor_str == '':
        return np.nan
    # Remove símbolos monetários e espaços
    valor_str = re.sub(r'[\$\s]', '', valor_str)
    # Remove vírgulas (separadores de milhar)
    valor_str = valor_str.replace(',', '')
    try:
        return float(valor_str)
    except ValueError:
        return np.nan

def converter_percentual(valor):
    """
    Converte percentual string para float (0-1).
    Exemplos: '95%' → 0.95, '' → NaN
    """
    if pd.isna(valor):
        return np.nan
    valor_str = str(valor).strip()
    if valor_str == '' or valor_str == '%':
        return np.nan
    # Remove símbolo %
    valor_str = valor_str.replace('%', '').strip()
    try:
        return float(valor_str) / 100
    except ValueError:
        return np.nan

def converter_booleano(valor):
    """
    Converte valores t/f, true/false, yes/no para boolean.
    """
    if pd.isna(valor):
        return np.nan
    valor_str = str(valor).strip().lower()
    if valor_str in ['t', 'true', 'yes', 'y', '1']:
        return True
    elif valor_str in ['f', 'false', 'no', 'n', '0']:
        return False
    else:
        return np.nan

def remover_linhas_totalmente_vazias(df):
    """Remove linhas onde todas as colunas são NaN."""
    return df.dropna(how='all').reset_index(drop=True)

print('Funções utilitárias carregadas com sucsso')

Funções utilitárias carregadas com sucesso


In [ ]:
# =============================================================================
# CARREGAMENTO DOS CSVs BRUTOS
# =============================================================================

print('\n[1/4] CARREGANDO DADOS BRUTOS...')

# Carrega com low_memory=False para evitar tipo misto em colunas
df_calendar = pd.read_csv(RAW_DIR / 'dados_brutos_calendar.csv', low_memory=False)
df_listings = pd.read_csv(RAW_DIR / 'dados_brutos_listings.csv', low_memory=False)
df_neighbourhoods = pd.read_csv(RAW_DIR / 'dados_brutos_neighbourhoods.csv', low_memory=False)
df_reviews = pd.read_csv(RAW_DIR / 'dados_brutos_reviews.csv', low_memory=False)

print(f'  calendar:      {df_calendar.shape[0]} linhas × {df_calendar.shape[1]} colunas')
print(f'  listings:      {df_listings.shape[0]} lihas × {df_listings.shape[1]} colunas')
print(f'  neighbourhoods: {df_neighbourhoods.shape[0]} linhas × {df_neighbourhoods.shape[1]} colunas')
print(f'  reviews:       {df_reviews.shape[0]} linhas × {df_reviews.shape[1]} colunas')


[1/4] CARREGANDO DADOS BRUTOS...
  calendar:      1048575 linhas × 4 colunas
  listings:      5764 linhas × 63 colunas
  neighbourhoods: 44 linhas × 1 colunas
  reviews:       62976 linhas × 10 colunas


In [ ]:
# =============================================================================
# [DADOS BRUTOS] → [SILVER] CALENDAR
# =============================================================================

print('\n[2/4] LIMPEZA: DADOS_BRUTOS_CALENDAR.CSV')

df_cal = df_calendar.copy()
linhas_inicial = len(df_cal)

# Padronizar nomes de colunas para snake_case
df_cal.columns = [converter_para_snake_case(col) for col in df_cal.columns]
print(f'  Colunas renomeadas: {list(df_cal.columns)}')

# Remover linhas totalmente vazias
df_cal = remover_linhas_totalmente_vazias(df_cal)
linhas_apos_vazias = len(df_cal)
print(f'  Linhas totalmente vazias removidas: {linhas_inicial - linhas_apos_vazias}')

# Converter 'date' para datetime
df_cal['date'] = pd.to_datetime(df_cal['date'], errors='coerce')
print(f'  Coluna "date" convertida para datetime')

# Converter 'available' (t/f) para boolean
df_cal['available'] = df_cal['available'].apply(converter_booleano)
print(f'  Coluna "available" convertida para boolean')

# Converter 'price' para float
df_cal['price'] = df_cal['price'].apply(converter_preco)
print(f'  Coluna "price" convertida para float')

# Remover duplicatas com base em (listing_id, date)
df_cal = df_cal.drop_duplicates(subset=['listing_id', 'date'], keep='first')
print(f'  Duplicatas removidas (listing_id, date)')

#  Verificações finais
nulos_price = df_cal['price'].isna().sum()
print(f'  Valores NaN em "price": {nulos_price}')
print(f'  Shape final: {df_cal.shape[0]} linhas × {df_cal.shape[1]} colunas')

# Salva resultado
output_path = SILVER_DIR / 'calendar_silver.csv'
df_cal.to_csv(output_path, index=False)
print(f'  ✓ Salvo: {output_path}')


[2/4] LIMPEZA: DADOS_BRUTOS_CALENDAR.CSV
  Colunas renomeadas: ['listing_id', 'date', 'available', 'price']
  Linhas totalmente vazias removidas: 0
  Coluna "date" convertida para datetime
  Coluna "available" convertida para boolean
  Coluna "price" convertida para float
  Duplicatas removidas (listing_id, date)
  Valores NaN em "price": 273502
  Shape final: 1048575 linhas × 4 colunas
  ✓ Salvo: ../silver/calendar_silver.csv


In [18]:
# =============================================================================
# [DADOS BRUTOS] → [SILVER] LISTINGS
# =============================================================================

print('\n LIMPEZA: DADOS_BRUTOS_LISTINGS.CSV')

df_list = df_listings.copy()
linhas_inicial = len(df_list)

#  Padronizar nomes de colunas para snake_case
df_list.columns = [converter_para_snake_case(col) for col in df_list.columns]
print(f'  Colunas renomeadas para snake_case ({len(df_list.columns)} colunas)')

#  Remover linhas totalmente vazias
df_list = remover_linhas_totalmente_vazias(df_list)
linhas_apos_vazias = len(df_list)
print(f'  Linhas totalmente vazias removidas: {linhas_inicial - linhas_apos_vazias}')

#  Limpeza de campos textuais (espaços extras)
colunas_texto = df_list.select_dtypes(include='object').columns
for col in colunas_texto:
    df_list[col] = df_list[col].apply(limpar_texto)
print(f'  Limpeza de texto aplicada em {len(colunas_texto)} colunas')

# Conversão de datas
colunas_datas = ['host_since', 'first_review', 'last_review']
for col in colunas_datas:
    if col in df_list.columns:
        df_list[col] = pd.to_datetime(df_list[col], errors='coerce')
        print(f'      "{col}" convertido para datetime')

#  Conversão de booleanos
colunas_booleanos = ['host_is_superhost', 'host_identity_verified', 'instant_bookable']
for col in colunas_booleanos:
    if col in df_list.columns:
        df_list[col] = df_list[col].apply(converter_booleano)
        print(f'      "{col}" convertido para boolean')

#  Conversão de preços (remove $, vírgulas)
colunas_preco = ['price', 'security_deposit', 'cleaning_fee', 'extra_people']
for col in colunas_preco:
    if col in df_list.columns:
        df_list[col] = df_list[col].apply(converter_preco)
        print(f'      "{col}" convertido para float')

#  Conversão de percentuais
colunas_perc = ['host_response_rate', 'host_acceptance_rate']
for col in colunas_perc:
    if col in df_list.columns:
        df_list[col] = df_list[col].apply(converter_percentual)
        print(f'      "{col}" convertido para float (0-1)')

#  Converter zipcode para string
if 'zipcode' in df_list.columns:
    df_list['zipcode'] = df_list['zipcode'].astype(str)
    print(f'  "zipcode" convertido para string')

# Remover registros com valores inconsistentes (outliers)
linhas_apos_limpeza = len(df_list)

# price <= 0 ou NaN
if 'price' in df_list.columns:
    df_list = df_list[(df_list['price'] > 0) | (df_list['price'].isna())]

# bathrooms <= 0
if 'bathrooms' in df_list.columns:
    df_list = df_list[(df_list['bathrooms'] > 0) | (df_list['bathrooms'].isna())]

# accommodates <= 0
if 'accommodates' in df_list.columns:
    df_list = df_list[df_list['accommodates'] > 0]

linhas_apos_outliers = len(df_list)
print(f'  Outliers removidos: {linhas_apos_limpeza - linhas_apos_outliers}')

# Remover duplicatas (por id)
if 'id' in df_list.columns:
    df_list = df_list.drop_duplicates(subset=['id'], keep='first')
    print(f'  Duplicatas removidas (id)')
else:
    df_list = df_list.drop_duplicates(keep='first')
    print(f'  Duplicatas removidas (todas as colunas)')

print(f'  Shape final: {df_list.shape[0]} linhas × {df_list.shape[1]} colunas')

# Salva resultado
output_path = SILVER_DIR / 'listings_silver.csv'
df_list.to_csv(output_path, index=False)
print(f'   Salvo: {output_path}')


 LIMPEZA: DADOS_BRUTOS_LISTINGS.CSV
  Colunas renomeadas para snake_case (63 colunas)
  Linhas totalmente vazias removidas: 0
  Limpeza de texto aplicada em 34 colunas
      "host_since" convertido para datetime
      "first_review" convertido para datetime
      "last_review" convertido para datetime
      "host_is_superhost" convertido para boolean
      "host_identity_verified" convertido para boolean
      "instant_bookable" convertido para boolean
      "price" convertido para float
      "security_deposit" convertido para float
      "cleaning_fee" convertido para float
      "extra_people" convertido para float
      "host_response_rate" convertido para float (0-1)
      "host_acceptance_rate" convertido para float (0-1)
  "zipcode" convertido para string
  Outliers removidos: 19
  Duplicatas removidas (id)
  Shape final: 5745 linhas × 63 colunas
   Salvo: ../silver/listings_silver.csv


In [ ]:
# =============================================================================
# [DADOS BRUTOS] → [SILVER] NEIGHBOURHOODS
# =============================================================================

print('\n[4/4] LIMPEZA: DADOS_BRUTOS_NEIGHBOURHOODS.CSV')

df_neigh = df_neighbourhoods.copy()
linhas_inicial = len(df_neigh)

# Padronizar nomes de colunas para snake_case
df_neigh.columns = [converter_para_snake_case(col) for col in df_neigh.columns]
print(f'  Colunas renomeadas: {list(df_neigh.columns)}')

#  Remover linhas totalmente vazias
df_neigh = remover_linhas_totalmente_vazias(df_neigh)
linhas_apos_vazias = len(df_neigh)
print(f'  Linhas totalmente vazias removidas: {linhas_inicial - linhas_apos_vazias}')

#  Limpeza de texto em colunas textuais
colunas_texto = df_neigh.select_dtypes(include='object').columns
for col in colunas_texto:
    df_neigh[col] = df_neigh[col].apply(limpar_texto)
print(f'  Limpeza de texto aplicada')

#  Garantir que 'neighbourhood' seja string
if 'neighbourhood' in df_neigh.columns:
    df_neigh['neighbourhood'] = df_neigh['neighbourhood'].astype(str)
    print(f'  "neighbourhood" convertido para string')

#  Remover duplicatas
df_neigh = df_neigh.drop_duplicates(keep='first').reset_index(drop=True)
print(f'  Duplicatas removidas')

print(f'  Shape final: {df_neigh.shape[0]} linhas × {df_neigh.shape[1]} colunas')

# Salva resultado
output_path = SILVER_DIR / 'neighbourhoods_silver.csv'
df_neigh.to_csv(output_path, index=False)
print(f'  ✓ Salvo: {output_path}')

# =============================================================================
# [DADOS BRUTOS] → [SILVER] REVIEWS
# =============================================================================

print('\n[5/5] LIMPEZA: DADOS_BRUTOS_REVIEWS.CSV')

df_rev = df_reviews.copy()
linhas_inicial = len(df_rev)

# --- Passo 1: Remover colunas totalmente vazias
colunas_vazias = df_rev.columns[df_rev.isna().all()].tolist()
df_rev = df_rev.drop(columns=colunas_vazias)
if colunas_vazias:
    print(f'  Colunas totalmente vazias removidas: {colunas_vazias}')
else:
    print(f'  Nenhuma coluna totalmente vazia encontrada')

# Padronizar nomes de colunas para snake_case
df_rev.columns = [converter_para_snake_case(col) for col in df_rev.columns]
print(f'  Colunas renomeadas: {list(df_rev.columns)}')

# Remover linhas totalmente vazias
df_rev = remover_linhas_totalmente_vazias(df_rev)
linhas_apos_vazias = len(df_rev)
print(f'  Linhas totalmente vazias removidas: {linhas_inicial - linhas_apos_vazias}')

# Limpeza de texto em colunas textuais
colunas_texto = df_rev.select_dtypes(include='object').columns
for col in colunas_texto:
    df_rev[col] = df_rev[col].apply(limpar_texto)
print(f'  Limpeza de texto aplicada em {len(colunas_texto)} colunas')

#  Converter 'date' para datetime
if 'date' in df_rev.columns:
    df_rev['date'] = pd.to_datetime(df_rev['date'], errors='coerce')
    print(f'  Coluna "date" convertida para datetime')

#  Remover duplicatas
df_rev = df_rev.drop_duplicates(keep='first').reset_index(drop=True)
print(f'  Duplicatas removidas')

print(f'  Shape final: {df_rev.shape[0]} linhas × {df_rev.shape[1]} colunas')

# Salva resultado
output_path = SILVER_DIR / 'reviews_silver.csv'
df_rev.to_csv(output_path, index=False)
print(f'  ✓ Salvo: {output_path}')


[4/4] LIMPEZA: DADOS_BRUTOS_NEIGHBOURHOODS.CSV
  Colunas renomeadas: ['neighbourhood']
  Linhas totalmente vazias removidas: 0
  Limpeza de texto aplicada
  "neighbourhood" convertido para string
  Duplicatas removidas
  Shape final: 44 linhas × 1 colunas
  ✓ Salvo: ../silver/neighbourhoods_silver.csv

[5/5] LIMPEZA: DADOS_BRUTOS_REVIEWS.CSV
  Nenhuma coluna totalmente vazia encontrada
  Colunas renomeadas: ['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'unnamed_5', 'unnamed_6', 'unnamed_7', 'unnamed_8', 'unnamed_9']
  Linhas totalmente vazias removidas: 0
  Limpeza de texto aplicada em 7 colunas
  Coluna "date" convertida para datetime
  Duplicatas removidas
  Shape final: 62976 linhas × 10 colunas
  ✓ Salvo: ../silver/reviews_silver.csv


In [15]:
# =============================================================================
# RESUMO E VALIDAÇÕES FINAIS
# =============================================================================

print('\n' + '=' * 70)
print('RESUMO FINAL DA LIMPEZA')
print('=' * 70)

print('\nESTATÍSTICAS DOS ARQUIVOS SILVER:')
print(f'\n  calendar_silver.csv')
print(f'      Linhas: {df_cal.shape[0]}')
print(f'      Colunas: {df_cal.shape[1]}')
print(f'      Nulos em price: {df_cal["price"].isna().sum()} ({df_cal["price"].isna().sum()/len(df_cal)*100:.2f}%)')
print(f'      Range de datas: {df_cal["date"].min()} até {df_cal["date"].max()}')

print(f'\n  listings_silver.csv')
print(f'      Linhas: {df_list.shape[0]}')
print(f'      Colunas: {df_list.shape[1]}')
colunas_com_nulos = df_list.isna().sum()
colunas_com_nulos = colunas_com_nulos[colunas_com_nulos > 0].sort_values(ascending=False)
if len(colunas_com_nulos) > 0:
    print(f'      Colunas com dados faltantes:')
    for col, count in colunas_com_nulos.head(5).items():
        print(f'          - {col}: {count} ({count/len(df_list)*100:.2f}%)')
else:
    print(f'      Sem dados faltantes críticos')

print(f'\n  neighbourhoods_silver.csv')
print(f'      Linhas: {df_neigh.shape[0]}')
print(f'      Colunas: {df_neigh.shape[1]}')

print(f'\n  reviews_silver.csv')
print(f'      Linhas: {df_rev.shape[0]}')
print(f'      Colunas: {df_rev.shape[1]}')
print(f'      Range de datas: {df_rev["date"].min()} até {df_rev["date"].max()}')

print('\nLIMPEZA CONCLUÍDA COM SUCESSO!')
print(f'Arquivos salvos em: {SILVER_DIR.resolve()}')
print('=' * 70)


RESUMO FINAL DA LIMPEZA

ESTATÍSTICAS DOS ARQUIVOS SILVER:

  calendar_silver.csv
      Linhas: 1048575
      Colunas: 4
      Nulos em price: 273502 (26.08%)
      Range de datas: 2015-11-07 00:00:00 até 2016-11-05 00:00:00

  listings_silver.csv
      Linhas: 5745
      Colunas: 63
      Colunas com dados faltantes:
          - security_deposit: 3022 (52.60%)
          - cleaning_fee: 2204 (38.36%)
          - review_scores_accuracy: 2020 (35.16%)
          - review_scores_cleanliness: 2018 (35.13%)
          - review_scores_communication: 2018 (35.13%)

  neighbourhoods_silver.csv
      Linhas: 44
      Colunas: 1

  reviews_silver.csv
      Linhas: 62976
      Colunas: 10
      Range de datas: 2008-09-13 00:00:00 até 2015-11-07 00:00:00

LIMPEZA CONCLUÍDA COM SUCESSO!
Arquivos salvos em: /home/mateus/Área de trabalho/airbnb/SBD2-Austin-Airbnb/Data Layer/silver
